In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from lightgbm.callback import early_stopping 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#データ処理
train_path='/kaggle/input/titanic/train.csv'
test_path='/kaggle/input/titanic/test.csv'

#DataFrame化
train_data=pd.read_csv(train_path)
test_data=pd.read_csv(test_path)

#nullの確認
print(train_data.isnull().sum())
print(test_data.isnull().sum())

#nullの処理
mode_embarked = train_data['Embarked'].mode()[0]
train_data['Embarked'] = train_data['Embarked'].fillna(mode_embarked)

test_data['Fare'] = test_data.groupby(['Pclass','Embarked'])['Fare'].transform(lambda x: x.fillna(x.median()))

train_data['Age'] = train_data.groupby(['Pclass','Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
test_data['Age'] = train_data.groupby(['Pclass','Sex'])['Age'].transform(lambda x: x.fillna(x.median()))

#新しい特徴量
train_data['FamilySize']=train_data['SibSp']+train_data['Parch']+1
test_data['FamilySize']=train_data['SibSp']+train_data['Parch']+1


train_data['IsAlone'] = 0
train_data.loc[train_data['FamilySize'] == 1, 'IsAlone'] = 1
test_data['IsAlone'] = 0
test_data.loc[test_data['FamilySize'] == 1, 'IsAlone'] = 1

train_data['Deck'] = train_data['Cabin'].str[0].fillna('U')  # U = Unknown
test_data['Deck'] = test_data['Cabin'].str[0].fillna('U')

train_data['FarePerPerson'] = train_data['Fare'] / train_data['FamilySize']
test_data['FarePerPerson'] = test_data['Fare'] / test_data['FamilySize']

def simplify_title(title):
    if title in ['Mr', 'Mrs', 'Miss', 'Master']:
        return title
    else:
        return 'Other'

train_data['Title'] = train_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False).apply(simplify_title)
test_data['Title'] = test_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False).apply(simplify_title)

X=train_data.drop(['PassengerId','Survived','Name','Cabin','Ticket'],axis=1)
y=train_data['Survived']
test_X=test_data.drop(['PassengerId','Name','Cabin','Ticket'],axis=1)

X=pd.get_dummies(X)
test_X=pd.get_dummies(test_X)

test_X = test_X.reindex(columns=X.columns, fill_value=0)

train_X,val_X,train_y,val_y=train_test_split(X,y,test_size=0.2,random_state=1)

lgb_model = lgb.LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)
lgb_model.fit(train_X, train_y, eval_set=[(val_X, val_y)], eval_metric='binary_logloss', callbacks=[early_stopping(stopping_rounds=100, verbose=False)])
val_preds = lgb_model.predict(val_X)

feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': lgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n特徴量重要度:")
print(feature_importance.head(15))

print("Accuracy:", accuracy_score(val_y, val_preds))

lgb_model = lgb.LGBMClassifier(
    n_estimators=300,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)
lgb_model.fit(X, y,eval_set=[(X, y)], eval_metric='binary_logloss', callbacks=[early_stopping(stopping_rounds=100,verbose=False)])
test_preds = lgb_model.predict(test_X)

output = pd.DataFrame({'PassengerId': test_data.PassengerId,'Survived': test_preds})
output.to_csv('submission.csv', index=False)

print(test_preds[:20])

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
[LightGBM] [Info] Number of positive: 269, number of negative: 443
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002347 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 366
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 23
[LightGBM] [I

/usr/local/lib/python3.11/dist-packages/lightgbm/callback.py:347: UserWarning: Only training set found, disabling early stopping.
  _log_warning("Only training set found, disabling early stopping.")


[0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1]
